### Training a Sarcasm Detection Model using Bidirectional LSTMs

In [1]:
#importing Libraries

import json
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
file_path="D:\\Tensorflow_Works\\8-Sentiment_in_Text\\sarcasm.json"

with open(file_path, "r") as file:
    data = json.load(file)

data[0] 

{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
 'is_sarcastic': 0}

In [3]:
sentences = []
labels = []
for element in data:
    sentences.append(element['headline'])
    labels.append(element['is_sarcastic'])

In [7]:
#setting parameters to easily tweak them when needed

#number of examples to use for training
training_size = 20000

#vocabulary size of tokenizer
vocab_size=10000

#maximum length of the padded sequences
max_length = 32

#output dimensions of the embedding layer
embedding_dims = 16

In [8]:
#splitting into train and test dataset
train_sentences = sentences[:training_size]
train_labels = labels[:training_size]

test_sentences = sentences[training_size:]
test_labels = labels[training_size:]

In [10]:
#defining vectorize layer

vectorize_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)

#generating vocabulary using training data

vectorize_layer.adapt(train_sentences)

In [13]:
def preprocessing(sequences):

    sequences = sequences.ragged_batch(batch_size=sequences.cardinality())

    sequences = sequences.get_single_element()

    padded_sequence = tf.keras.utils.pad_sequences(sequences.numpy(), padding="pre", truncating="post", maxlen=max_length)

    padded_sequence = tf.data.Dataset.from_tensor_slices(padded_sequence)

    return padded_sequence


In [20]:
pad_train_sentences = preprocessing(tf.data.Dataset.from_tensor_slices(vectorize_layer(train_sentences)))
pad_test_sentences = preprocessing(tf.data.Dataset.from_tensor_slices(vectorize_layer(test_sentences)))


In [21]:
train_labels_dataset = tf.data.Dataset.from_tensor_slices(train_labels)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(test_labels)

In [22]:
train_dataset = tf.data.Dataset.zip(pad_train_sentences, train_labels_dataset)
test_dataset = tf.data.Dataset.zip(pad_test_sentences, test_labels_dataset)